# Final Project Foundamentals of Data Science
### Solar Eclipse phases classification
---------



### Abstract


In [6]:
if True:
    %pip install -qqq numpy scipy matplotlib pandas scikit-learn seaborn tqdm

Note: you may need to restart the kernel to use updated packages.


In [16]:
%matplotlib inline
import numpy as np # imports a fast numerical programming library
import matplotlib.pyplot as plt # sets up plotting under plt
import matplotlib.image as mpimg
import pandas as pd # lets us handle data as dataframes
import seaborn as sns
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import os
from pathlib import Path
%load_ext autoreload
%autoreload 2

# sets up pandas table display
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### 1. Analisi del dataset

The “Great American Eclipse” in 2017 offered a unique opportunity for US citizens across the entire continent to experience the awe-inspiring changes that occur during rare total solar eclipses (TSEs). The goal of the 2017 Eclipse Megamovie was to obtain enough pictures of the Great American Total Solar Eclipse to create a movie of the solar corona in order to better study it. Over 2,000 volunteers submitted 50,000 images and a movie was made only a few hours after the Moon's shadow left the U.S. A few months later another, better aligned and more closely edited, movie was released. The Eclipse Megamovie team of scientists processed images to create High Dynamic Range (HDR) images to capture the structures within the high dynamic range of the solar corona from the Sun's photosphere out ~3 solar radii.
https://eclipsemegamovie.org

Eclipse megamovie è un progetto che viene portato avanti da un gruppo di ricercatori in collaborazione con la NASA. Il punto centrale di tale progetto è quello di realizzare un video in grado di facilitare la ricerca su un evento raro come l'eclissi. Per realizzare questo progetto è stato chiesto ai cittadini americani di inviare fotografie scattate al sole durante l'eclissi. Il dataset contiene quindi immagini semi-professionali e amatoriali, scattate in condizioni non sempre ottimali. 

Eclipse megamovie is a project being carried out by a group of researchers in collaboration with NASA. The focus of this project is to make a video that can facilitate research into a rare event such as the eclipse that occurred in the United States in 2017. To realise this project, American citizens were asked to send in photographs taken during the eclipse. The dataset therefore contains semi-professional and amateur images, taken under conditions that are not always optimal.

In [ ]:
# Carica le immagini (sostituisci con i tuoi file)

desktop_path = Path.home() / "Desktop"
dataset_path = desktop_path / "eclupse"

img1 = mpimg.imread('/Users/valeriosantini/Desktop/eclipse-megamovie/train/00a0f91f0e7ece88ebaeab2a2050cdc0d72c5ea0b56c601dfaf800077df44c51.jpg')
img2 = mpimg.imread('/Users/valeriosantini/Desktop/eclipse-megamovie/train/00a4af4cc3f2a667c80331051a733e8dae2477d682d7dcef3e895f6f923dd86a.jpg')
img3 = mpimg.imread('/Users/valeriosantini/Desktop/eclipse-megamovie/train/00c11b16fd6426a88039c8b27e2511e2e770b65e0eb9daab720d3e6ca996bc06.jpg')
img4 = mpimg.imread('/Users/valeriosantini/Desktop/eclipse-megamovie/train/001e6704a60798684f1656713b593e6a56de81474e7173a31936589cd62f13e8.jpg')

# Imposta una figura con sottotrame affiancate
fig, axes = plt.subplots(1, 4, figsize=(16, 4))  # 1 riga, 4 colonne, dimensioni regolate
fig.tight_layout(pad=3.0)  # Regola la spaziatura tra le immagini

# Mostra ciascuna immagine in una sottotrama
axes[0].imshow(img1)
axes[0].set_title("Image 1")
axes[0].axis('off')  # Nasconde gli assi

axes[1].imshow(img2)
axes[1].set_title("Image 2")
axes[1].axis('off')

axes[2].imshow(img3)
axes[2].set_title("Image 3")
axes[2].axis('off')

axes[3].imshow(img4)
axes[3].set_title("Image 4")
axes[3].axis('off')

# Mostra la griglia di immagini
plt.show()


FileNotFoundError: [Errno 2] No such file or directory: '/Users/valeriosantini/Documents/GitHub/Corasoft-Unity/FDS/FInal Project/eclipse_megamovie/train/00a0f91f0e7ece88ebaeab2a2050cdc0d72c5ea0b56c601dfaf800077df44c51.jpg'